# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
pip install pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
import pandas as pd
import json
import pandas as pd  


In [5]:
# Specify the path to your JSON file
json_file_path = 'establishments.json'

# Read the JSON file into a Pandas DataFrame
df = pd.read_json(json_file_path)

# Now 'df' contains the data from your JSON file as a DataFrame

In [6]:

db = mongo['uk_food']

# Access the 'establishments' collection
collection = db['establishments']

In [7]:
# confirm that our new database was created
# Find and display one document from the 'establishments' collection
document = collection.find_one()

# Display the document using pprint
pprint(document)


{'AddressLine1': '146A Plumstead Rd',
 'AddressLine2': 'Penang Flavours',
 'AddressLine3': 'London',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': None,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'PostCode': 'SE18 7DY',
 'RatingValue': {'$toInt': '$RatingValue'},
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('659b5a6cd17d675675d970ee'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [8]:
document_count = collection.count_documents({})
print(f"Number of documents in 'establishments' collection: {document_count}")


Number of documents in 'establishments' collection: 1


In [9]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [10]:
# review the collections in our new database

# Access the 'uk_food' database
db = mongo['uk_food']

# List collections in the 'uk_food' database
collection_names = db.list_collection_names()

# Print the collection names
print("Collections in 'uk_food' database:")
for collection_name in collection_names:
    print(collection_name)



Collections in 'uk_food' database:
establishments


In [11]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [13]:
# Create a dictionary for the new restaurant data
new_restaurant_info = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine1": "146A Plumstead Rd",
    "AddressLine2": "Penang Flavours",
    "AddressLine3": "London",
    "PostCode": "SE18 7DY",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {"Hygiene": "", "Structural": "", "ConfidenceInManagement": ""},
    "SchemeType": "FHRS",
    "geocode": {"longitude": "0.08384000", "latitude": "51.49014200"},
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [14]:
# Insert the new restaurant into the collection
new_restaurant_id = db['establishments'].insert_one(new_restaurant_info).inserted_id
print(f"New restaurant added with ID: {new_restaurant_id}")

New restaurant added with ID: 659b654c021ef5dc4725690a


In [ ]:
# Check that the new restaurant was inserted


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [15]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_info = db['establishments'].find_one({"BusinessType": "Restaurant/Cafe/Canteen"}, {"BusinessTypeID": 1, "BusinessType": 1})
pprint(business_type_info)


{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': None,
 '_id': ObjectId('659b5a6cd17d675675d970ee')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [16]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = business_type_info.get("BusinessTypeID")
db['establishments'].update_one({"_id": new_restaurant_id}, {"$set": {"BusinessTypeID": business_type_id}})

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
# Confirm that the new restaurant was updated


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [17]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_documents_count = db['establishments'].count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with Dover Local Authority: {dover_documents_count}")


Number of documents with Dover Local Authority: 0


In [18]:
# Delete all documents where LocalAuthorityName is "Dover"
db['establishments'].delete_many({"LocalAuthorityName": "Dover"})


DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [19]:
# Check if any remaining documents include Dover
remaining_documents_count = db['establishments'].count_documents({})
print(f"Number of remaining documents: {remaining_documents_count}")


Number of remaining documents: 2


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [22]:
# Change the data type from String to Integer for RatingValue
db['establishments'].update_many({}, {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}, "geocode.latitude": {"$toDouble": "$geocode.latitude"}}})

db['establishments'].update_many({}, {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}})

UpdateResult({'n': 2, 'nModified': 0, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)